In [1]:
""" Process unimproved/no drinking water and store on BigQuery.
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20181205
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

SCRIPT_NAME = "Y2018M12D05_RH_UDW_BQ_V01"
OUTPUT_VERSION = 1

NODATA_VALUE = -9999

S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/finalData/AccessToDW"
INPUT_FILE_NAME = "dw_results.csv"

BQ_PROJECT_ID = "aqueduct30"
BQ_OUTPUT_DATASET_NAME = "aqueduct30v01"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 

print("S3_INPUT_PATH: ",S3_INPUT_PATH,
      "\nec2_input_path: ",ec2_input_path,
      "\nec2_output_path: ",ec2_output_path,
      "\nBQ_OUTPUT_DATASET_NAME: ", BQ_OUTPUT_DATASET_NAME,
      "\nBQ_OUTPUT_TABLE_NAME: ",BQ_OUTPUT_TABLE_NAME
      )

S3_INPUT_PATH:  s3://wri-projects/Aqueduct30/finalData/AccessToDW 
ec2_input_path:  /volumes/data/Y2018M12D05_RH_UDW_BQ_V01/input_V01 
ec2_output_path:  /volumes/data/Y2018M12D05_RH_UDW_BQ_V01/output_V01 
BQ_OUTPUT_DATASET_NAME:  aqueduct30v01 
BQ_OUTPUT_TABLE_NAME:  y2018m12d05_rh_udw_bq_v01_v01


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M12D05 UTC 13:37


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

In [4]:
!aws s3 cp {S3_INPUT_PATH} {ec2_input_path} --recursive 

download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/dw_results.CPG to ../../../../data/Y2018M12D05_RH_UDW_BQ_V01/input_V01/dw_results.CPG
download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/dw_results.cpg to ../../../../data/Y2018M12D05_RH_UDW_BQ_V01/input_V01/dw_results.cpg
download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/README.txt to ../../../../data/Y2018M12D05_RH_UDW_BQ_V01/input_V01/README.txt
download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/dw_results.prj to ../../../../data/Y2018M12D05_RH_UDW_BQ_V01/input_V01/dw_results.prj
download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/dw_results.sbx to ../../../../data/Y2018M12D05_RH_UDW_BQ_V01/input_V01/dw_results.sbx
download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/dw_results.shp.xml to ../../../../data/Y2018M12D05_RH_UDW_BQ_V01/input_V01/dw_results.shp.xml
download: s3://wri-projects/Aqueduct30/finalData/AccessToDW/wri_aqueduct3_drinkingwater.py to ../../../../data/Y2018M12D05_R

In [5]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

In [6]:
files = os.listdir(ec2_input_path)

In [7]:
input_path = "{}/{}".format(ec2_input_path,INPUT_FILE_NAME)

In [8]:
df = pd.read_csv(input_path)

In [9]:
df.dtypes

PFAF_ID         int64
DW_nat_raw    float64
DW_rur_raw    float64
DW_urb_raw    float64
rur_pop       float64
urb_pop       float64
total_pop     float64
DW_raw        float64
DW_s          float64
DW_cat         object
dtype: object

In [10]:
df.head()

,PFAF_ID,DW_nat_raw,DW_rur_raw,DW_urb_raw,rur_pop,urb_pop,total_pop,DW_raw,DW_s,DW_cat
0,111011,0.011696,0.016651,0.005164,67212.864470,60229.01666,127441.881100,0.011222,0.000000,Low (< 2.5%)
1,111012,0.011696,0.016651,0.005164,31891.179690,0.00000,31891.179690,0.016651,0.413638,Low (< 2.5%)
2,111013,0.011696,0.016651,0.005164,5741.101992,0.00000,5741.101992,0.016651,0.413638,Low (< 2.5%)
3,111014,0.011696,0.016651,0.005164,6491.622601,0.00000,6491.622601,0.016651,0.413638,Low (< 2.5%)
4,111015,0.011696,0.016651,0.005164,394618.164000,0.00000,394618.164000,0.016651,0.413638,Low (< 2.5%)


In [11]:
# DW -> UDW


# raw -> raw.
# s -> score.
# None -> cat.
# cat -> label. 

In [12]:
df_out = df.rename(columns={"PFAF_ID":"pfaf_id",
                            "DW_raw":"udw_raw",
                            "DW_s":"udw_score",
                            "DW_cat":"udw_label"})

In [13]:
df_out.drop(columns=["DW_nat_raw",
                     "DW_rur_raw",
                     "DW_urb_raw",
                     "rur_pop",
                     "urb_pop",
                     "total_pop"],inplace=True)

In [14]:
df_out["udw_raw"] = df_out["udw_raw"].fillna(NODATA_VALUE)
df_out["udw_score"] = df_out["udw_score"].fillna(NODATA_VALUE)
df_out["udw_label"] = df_out["udw_label"].fillna("No Data")

In [15]:
def score_to_category(score):
    if score != 5:
        cat = int(np.floor(score))
    else:
        cat = 4
    return cat

In [16]:
df_out["udw_cat"] = df_out["udw_score"].apply(score_to_category)


In [17]:
df_out = df_out.reindex(sorted(df_out.columns), axis=1)

In [18]:
destination_table = "{}.{}".format(BQ_OUTPUT_DATASET_NAME,BQ_OUTPUT_TABLE_NAME)

In [19]:
df.to_gbq(destination_table=destination_table,
          project_id=BQ_PROJECT_ID,
          chunksize=10000,
          if_exists="replace")

2it [00:04,  2.05s/it]


In [20]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:00:18.328004


Previous runs:  
0:00:18.766466